In [1]:
# this script is based on https://github.com/jianxu305/nCov2019_analysis. Credit the formatted data to him!
import pandas as pd
import os
import numpy as np

In [2]:
_CHN_EN_DICT_ = './data/locationDict.csv'
_DXY_DATA_PATH_ = 'https://raw.githubusercontent.com/BlankerL/DXY-2019-nCoV-Data/master/csv/DXYArea.csv'

In [12]:
def load_chinese_raw(path):
    '''
    This provides a way to look into the 'raw' data. If path is None, we fetch it from internet.
    '''
    if path == None:
        path = _DXY_DATA_PATH_
    raw = pd.read_csv(path)
    
    # the original CSV column names are in camel case, change to lower_case convention
    rename_dict = {'updateTime': 'update_time',
                   'provinceName': 'province_name',
                   'cityName': 'city_name',
                   'province_confirmedCount': 'province_confirmed',
                   'province_suspectedCount': 'province_suspected',
                   'province_deadCount': 'province_dead',
                   'province_curedCount': 'province_cured',
                   'city_confirmedCount': 'city_confirmed',
                   'city_suspectedCount': 'city_suspected',
                   'city_deadCount': 'city_dead',
                   'city_curedCount': 'city_cured',
                   #'provinceEnglishName': 'province_name_en',
                   #'cityEnglishName': 'city_name_en'
                  }
    # drop zip columns
    raw = raw.drop(columns = ['province_zipCode', 'city_zipCode'])
    # temprorarily drop English name columns
    raw = raw.drop(columns = ['provinceEnglishName', 'cityEnglishName'])
    data = raw.rename(columns=rename_dict)
    data['update_time'] = pd.to_datetime(data['update_time'])  # original type of update_time after read_csv is 'str'
    data['update_date'] = data['update_time'].dt.date    # add date for daily aggregation, if without to_datetime, it would be a dateInt object, difficult to use
    # display basic info
    print('Last update: ', data['update_time'].max())
    print('Data date range: ', data['update_date'].min(), 'to', data['update_date'].max())
    print('Number of rows in raw data: ', data.shape[0])
    return data 

In [14]:
def rename_cities(data):
    '''
    Sometimes, for example 2/3/2020, on some time snapshots, the CSV data contains city_name entries such as "南阳", "商丘", but at other time snapshots, it contains "南阳（含邓州）",  and "商丘（含永城）", etc.  They should be treated as the same city
    This results in the aggregation on province level getting too high.
    For now, entries will be ignored if city_name == xxx(xxx), and xxx already in the city_name set
    '''
    dup_frm = data[data['city_name'].str.contains('（')]
    rename_dict = {name: name.split('（')[0] for name in dup_frm['city_name']}
    dupCounty_frm = data[data['city_name'].str.endswith('县')]
    dupCity_frm = data[data['city_name'].str.endswith('市')]
    for name in dupCounty_frm['city_name']:
        if len(name) >= 3: # for names with length 2 and ending with '县', like '滑县', no need to change
            rename_dict[name] = name.split('县')[0]
    for name in dupCity_frm['city_name']:
        if len(name) >= 3:
            rename_dict[name] = name.split('市')[0]
    rename_dict['吐鲁番市'] = '吐鲁番'   # raw data has both 吐鲁番市 and 吐鲁番, should be combined
    rename_dict['虹口'] = '虹口区'
    rename_dict['嘉定'] = '嘉定区'
    rename_dict['浦东'] = '浦东新区'
    rename_dict['黄浦'] = '黄浦区'
    rename_dict['杨浦'] = '杨浦区'
    
    rename_dict['浦东区'] = '浦东新区'
    rename_dict['丰台'] = '丰台区'
    rename_dict['静安'] = '静安区'
    rename_dict['青浦'] = '青浦区'
    rename_dict['大兴'] = '大兴区'
    rename_dict['宝山'] = '宝山区'
    rename_dict['徐汇'] = '徐汇区'
    rename_dict['门头沟'] = '门头沟区'
    rename_dict['闵行'] = '闵行区'
    rename_dict['东城'] = '东城区'
    rename_dict['通州'] = '通州区'
    rename_dict['奉贤'] = '奉贤区'
    rename_dict['红河州'] = '红河'
    rename_dict['楚雄州'] = '楚雄'
    rename_dict['大理州'] = '大理'
    rename_dict['文山州'] = '文山'
    rename_dict['德宏州'] = '德宏'
    rename_dict['第八师石河子市'] = '兵团第八师石河子市'
    rename_dict['第八师石河子'] = '兵团第八师石河子市'
    rename_dict['石河子'] = '兵团第八师石河子市'
    rename_dict['第八师'] = '兵团第八师石河子市'
    rename_dict['第七师'] = '兵团第八师'
    rename_dict['呼伦贝尔牙克石市'] = '呼伦贝尔牙克石'
    rename_dict['通辽市经济开发区'] = '通辽'
    rename_dict['第九师'] = '兵团第九师'
    rename_dict['西双版纳州'] = '西双版纳'
    rename_dict['滑县'] = ['滑县']
    rename_dict['待明确'] = '待明确地区'
    rename_dict['未明确地区'] = '待明确地区'
    rename_dict['未知'] = '待明确地区'
    rename_dict['未明确'] = '待明确地区'
    rename_dict['未知地区'] = '待明确地区'
    
    data['city_name'] = data['city_name'].replace(rename_dict)  # write back
    return data

In [5]:
def rename_provinces(data):
    '''
    For simplicity, we only use simple names of provinces. For example, '河南省' will be renamed '河南'; '广西壮族自治区' will be renamed '广西'
    '''
    dup_frm = data[data['province_name'].str.contains('市|省')]
    rename_dict = {name: name.split('市')[0] if '市' in name else name.split('省')[0] for name in dup_frm['province_name']}
    rename_dict['内蒙古自治区'] = '内蒙古'
    rename_dict['新疆维吾尔自治区'] = '新疆'
    rename_dict['宁夏回族自治区'] = '宁夏'
    rename_dict['广西壮族自治区'] = '广西'
    rename_dict['西藏藏族自治区'] = '西藏'
    rename_dict['西藏自治区'] = '西藏'
    data['province_name'] = data['province_name'].replace(rename_dict)
    return data

In [6]:
def add_en_location(df):
    '''Add province_name_en, and city_name_en, should only be called after calling rename_cities'''
    chn_en = pd.read_csv(_CHN_EN_DICT_, encoding='utf-8')
    translation = dict([t for t in zip(chn_en['Chinese'], chn_en['English'])])
    df['province_name_en'] = df['province_name'].replace(translation)
    df['city_name_en'] = df['city_name'].replace(translation)
    return df

In [7]:
def aggDaily(df):
    '''Aggregate the frequent time series data into a daily frame, ie, one entry per (date, province, city)'''
    frm_list = []
    for key, frm in df.sort_values(['update_date']).groupby(['province_name', 'city_name', 'update_date']):
        frm_list.append(frm.sort_values(['update_time'])[-1:])    # take the latest row within (city, date)
    out = pd.concat(frm_list).sort_values(['update_date', 'province_name', 'city_name'])
    out = out.drop(columns="update_time")
    out.sort_values(['province_name', 'city_name', 'update_date'], inplace=True)
    return out.set_index(['province_name', 'city_name', 'update_date'])

In [8]:
def load_virus_data(path = None):
    '''
    a pipline for load_chinese_raw, rename_cities add_en_location and aggDaily.
    If path is None, we fetch is from internet.
    Also, drop the entries with city_name '待明确地区'， '外地来*人员 for simplicity.
    '''
    data = load_chinese_raw(path)
    data = rename_cities(data)
    
    dropped_indices = data[(data['city_name'] == '待明确地区') | (data['city_name'].str.contains('外地来'))].index
    print("dropped {} rows with 待明确地区 or 外地来*人员 as city_name.".format(len(dropped_indices)))
    data.drop(dropped_indices, inplace = True)
    
    data = add_en_location(data)
    data = rename_provinces(data)
    return aggDaily(data)

In [17]:
def complete_missing(df, report = True):
    '''
    At some date there are missing data. We complete it using the data at previous date.
    If there are no available previous data, we set all numerical values 0.
    'report' controls whether a completion report is printed out, defaults to True.
    '''
    indices = df.index
    time_series = indices.get_level_values(2)
    province_indices = set(indices.get_level_values(0)) # get the indices of province level
    complete_time_indices = pd.date_range(time_series.min(), time_series.max(), freq='D') # the complete time indices
    numerical_columns = [region +'_'+ case for region in ['province', 'city'] for case in ['confirmed', 'suspected', 'cured', 'dead']]
    other_columns = set(df.columns) - set(numerical_columns) # other text based columns, such as English name
    updated_provinces = []
    num_completed = 0
    for province in province_indices:
        province_df = df.loc[province].copy() # get the sub dataframe for each province

        city_indices = set(province_df.index.get_level_values(0)) # get the indices of city level within this province
        updated_cities = []
        for city in city_indices:
            city_df = province_df.loc[city].copy() # get the sub dataframe for each city
            time_indices = city_df.index
            first = True # if the first date's data is missing, then this is used
            for i, date in enumerate(complete_time_indices):
                if date in time_indices:
                    first = False
                    continue

                # the date at this date is missing
                if first:
                    first_date = time_indices[0]
                    city_df.loc[date] = [0 if name in numerical_columns else city_df.loc[first_date, name] for name in city_df.columns]
                else:
                    city_df.loc[date] = city_df.loc[complete_time_indices[i - 1]].copy() # we copy it so that we can freely change one row without modifying others
                if report:
                    # messages printed got so long, change to a log file later
                    # print("\nmissing data at:", province, city, str(date.year) +'-'+ str(date.month) +'-'+ str(date.day))
                    # print("completed as:\n", city_df.loc[date].to_dict())
                    num_completed += 1
            city_df = city_df.sort_index(ascending = True)

            updated_cities.append(pd.concat([city_df], keys=[city], names=['city_name'])) # prepend the city-level index and append it to updated_cities
        updated_cities = pd.concat(updated_cities)
        updated_cities = updated_cities.sort_index(ascending = True, level = 0) # sort city_name, just for consistency
        updated_provinces.append(pd.concat([updated_cities], keys=[province], names=['province_name'])) # prepend the province-level index
    new_df = pd.concat(updated_provinces).sort_index(ascending=True, level=0) # sort province_name, just for consistency
    if report:
        total_rows = new_df.shape[0]
        print("completed {} rows out of {}".format(num_completed, total_rows))
    return new_df

In [10]:
def to_json(data, file_name, start_date = None, end_date = None):
    '''
    output cleaned data to a json file within the time range [start_date, end-date].
    start_date and end_date are two strings of format "year-month-day".
    If they are not provided, use possibly largest time range in the data instead.
    '''
    # sorry for messy code
    
    multi_indices = data.index
    time_series = multi_indices.get_level_values(2)
    complete_min = time_series.min()
    complete_max = time_series.max()
    start_date = complete_min if start_date == None else max(pd.Timestamp(start_date), complete_min)
    end_date = complete_max if end_date == None else min(pd.Timestamp(end_date), complete_max) # regularize start, end date
    if start_date > end_date:
        raise ValueError('start_date is larger than end_date')
    print('actual time range used: ', start_date, end_date)
    complete_time_indices = pd.date_range(start_date, end_date, freq='D') # the complete time indices
    province_indices = set(multi_indices.get_level_values(0))
    cases = ['confirmed', 'suspected', 'dead', 'cured']
    province_cases = ['province_' + case for case in cases]
    city_cases = ['city_' + case for case in cases]
    
    indent_level = 0
    newline = lambda file, num_tabs: file.write('\n' + ('\t' * num_tabs)) # used to begin a new line with prescribed indentation
    write_key_number = lambda file, key, number: file.write('"' + key + '": ' + str(number))
    write_key_string = lambda file, key, string: file.write('"' + key + '": "' + string + '"')
    write_key_date = lambda file, date: file.write('"time": "' + str(date.month) +'-'+ str(date.day) +'-'+ str(date.year) + '"')
    write_date_colon = lambda file, date: file.write('"' + str(date.month) +'-'+ str(date.day) +'-'+ str(date.year) + '": ')
    write_key_colon = lambda file, key: file.write('"' + key + '": ')

    with open(file_name, "w") as f:        
        f.write('{'); indent_level+=1; newline(f, indent_level)
        for pro_id, province in enumerate(province_indices):
            province_df = data.loc[province]
            province_en = province_df['province_name_en'][0]
            write_key_colon(f, province_en) # name of this province as the key
            f.write('{'); indent_level+=1; newline(f, indent_level) # begin a province
            
            
            write_key_string(f, "Chinese_name", str(province)) # the Chinese name of the province
            f.write(','); newline(f, indent_level)

            write_key_colon(f, "cases")
            f.write('['); indent_level+=1; newline(f, indent_level) # begin province-level cases
            
            for date_id, date in enumerate(complete_time_indices):
                f.write("{"); indent_level+=1; newline(f, indent_level) # begin each time frame
                write_key_date(f, date) # the time of each time frame
                f.write(','); newline(f, indent_level)

                for case_id, case in enumerate(province_cases):
                    write_key_number(f, cases[case_id], province_df[case][date_id]) # the count for this case at this date
                    if case_id < len(province_cases) - 1:
                        f.write(','); newline(f, indent_level) # prepare for the next case
                
                indent_level-=1; newline(f, indent_level); f.write("}") # end each time frame
                if date_id < len(complete_time_indices) - 1:
                    f.write(','); newline(f, indent_level) # prepare for next time frame
                   
            indent_level-=1; newline(f, indent_level); f.write(']') # end province-level cases
            f.write(','); newline(f, indent_level)
            
            write_key_colon(f, "cities")
            f.write('{'); indent_level+=1; newline(f, indent_level) # begin cities
            city_indices = set(province_df.index.get_level_values(0))
            for city_id, city in enumerate(city_indices):
                city_en = province_df.loc[city]['city_name_en'][0]
                write_key_colon(f, city_en) #  city name as the key
                f.write("{"); indent_level+=1; newline(f, indent_level) # begin each city
                
                write_key_string(f, 'Chinese_name', str(city)) # the Chinese name of this city
                f.write(','); newline(f, indent_level);
                write_key_colon(f, "cases")
                f.write('['); indent_level+=1; newline(f, indent_level) # begin city-level cases
                for date_id, date in enumerate(complete_time_indices):
                    f.write("{"); indent_level+=1; newline(f, indent_level) # begin each time frame
                    write_key_date(f, date) # the time of each time frame
                    f.write(','); newline(f, indent_level)
                    for case_id, case in enumerate(city_cases):
                        write_key_number(f, cases[case_id], province_df.loc[city][case][date_id]) # the count for this case at this date
                        if case_id < len(city_cases) - 1:
                            f.write(','); newline(f, indent_level) # prepare for the next case
                
                    indent_level-=1; newline(f, indent_level); f.write("}") # end each time frame
                    if date_id < len(complete_time_indices) - 1:
                        f.write(','); newline(f, indent_level) # prepare for next time frame
                indent_level-=1; newline(f, indent_level); f.write("]") # end city-level cases
                indent_level-=1; newline(f, indent_level); f.write("}") # end each city
                if city_id < len(city_indices) - 1:
                    f.write(','); newline(f, indent_level) # prepare for the next city

            indent_level-=1; newline(f, indent_level); f.write('}') # end cities
            indent_level-=1; newline(f, indent_level); f.write("}") # end a province
            if pro_id < len(province_indices) - 1:
                f.write(","); newline(f, indent_level)  # prepare for next province
                
        indent_level-=1; newline(f, indent_level); f.write('}')

In [15]:
data = load_virus_data('./data/DXYArea_new.csv')

Last update:  2020-02-23 09:07:39.744000
Data date range:  2020-01-24 to 2020-02-23
Number of rows in raw data:  51836
dropped 1006 rows with 待明确地区 or 外地来*人员 as city_name.


In [18]:
completed = complete_missing(data)

completed 2827 rows out of 14322


In [19]:
to_json(completed, './data/DXYArea_long.json')

actual time range used:  2020-01-24 00:00:00 2020-02-23 00:00:00


In [32]:
completed.loc['海南']['province_name_en']

city_name  update_date
万宁         2020-01-24     Henan
           2020-01-25     Henan
           2020-01-26     Henan
           2020-01-27     Henan
           2020-01-28     Henan
                          ...  
陵水         2020-02-09     Henan
           2020-02-10     Henan
           2020-02-11     Henan
           2020-02-12     Henan
           2020-02-13     Henan
Name: province_name_en, Length: 378, dtype: object

In [19]:
set(completed.loc['云南'].index.get_level_values(0))

{'临沧',
 '丽江',
 '丽江市',
 '保山',
 '大理',
 '大理州',
 '德宏',
 '德宏州',
 '文山',
 '文山州',
 '昆明',
 '昭通',
 '普洱',
 '曲靖',
 '楚雄',
 '楚雄州',
 '玉溪',
 '红河',
 '西双版纳'}